In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

2.4.1+cu121


In [ ]:
# !wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Test_Input.zip
# !wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Test_GroundTruth.zip
# !unzip ISIC2018_Task1-2_Test_Input.zip.1
# !unzip ISIC2018_Task1_Test_GroundTruth.zip

# !rm ISIC2018_Task1-2_Test_Input.zip.1
# !rm ISIC2018_Task1_Test_GroundTruth.zip
folder1 = "./CVC-ColonDB/images"
folder2 = "./CVC-ColonDB/masks"
# folder1 = "./ISIC2018_Task1-2_Test_Input"
# folder2 = "./ISIC2018_Task1_Test_GroundTruth"

images = sorted(os.listdir(folder1))
masks = sorted(os.listdir(folder2))

images = [folder1 + '/' + x for x in images]
masks = [folder2 + '/' + x for x in masks]

images = images[1:-1]
masks = masks[1:-1]

In [ ]:
len(masks), len(images)

(378, 378)

In [ ]:
for a,b in zip(masks,images):
  print(a,b)

In [ ]:
!pip install -q torch_geometric
!pip install -q class_resolver
!pip3 install pymatting

In [ ]:
from sklearn.metrics import jaccard_score

def iou(mask1, mask2):
    x = mask1.ravel()
    y = mask2.ravel()
    intersection = np.logical_and(x, y)
    union = np.logical_or(x, y)
    similarity = np.sum(intersection)/ np.sum(union)
    return similarity

In [ ]:
import numpy as np

def cts(image, segmap,wnd = [20,20]):
    """
    Crop the image and segmentation map to the boundary of the segmentation.

    :param image: A numpy array representing the color image.
    :param segmap: A numpy array representing the binary segmentation map.
    :return: Cropped image and segmentation map.
    """
    [ht,wdt] = segmap.shape
    # Find the indices where segmap is 1
    rows, cols = np.where(segmap == 255)
    # Find min and max coordinates
    min_row, max_row = max(min(rows)-wnd[0],0), min(max(rows)+wnd[0],ht)
    min_col, max_col = max(min(cols)-wnd[1],0), min(max(cols)+wnd[1],wdt)

    # Crop the image and segmap
    cropped_image = image[min_row:max_row+1, min_col:max_col+1]
    cropped_segmap = segmap[min_row:max_row+1, min_col:max_col+1]

    return cropped_image, cropped_segmap

In [ ]:
%load_ext autoreload
%autoreload 1

In [ ]:
%aimport segment
%aimport util
%aimport gnn_pool
%aimport sim

In [ ]:
epochs = [55]

# Number of clusters
K = 2

res = (224, 224)
cut = 0
mode = 0
bs = True #False
cc = False
stride = 4
facet = 'key'
layer = 11
pretrained_weights = './dino_deitsmall8_pretrain_full_checkpoint.pth'
#download the pretrained_weights from https://dl.fbaipublicfiles.com/dino/dino_deitsmall8_pretrain/dino_deitsmall8_pretrain_full_checkpoint.pth
in_dir = images
out_dir = './results_CVCColon_GAT'
log_bin = False
############# for cut==0, alpha will act like threshold
alpha = 0.5 #0.35 for ISIC2018
######################################################
save = True
activ = "SiLU_GAT" #"SiLU_GAT3" for ISIC2018
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from bilateral_solver import bilateral_solver_output
from features_extract import deep_features
from torch_geometric.data import Data
from extractor import ViTExtractor
import torch.optim as optim
from tqdm import tqdm
import numpy as np
import torch
import util
import os
from PIL import Image
from google.colab.patches import cv2_imshow
import cv2
import matplotlib.pyplot as plt
from numpy import asarray
import tifffile as tiff
from gnn_pool import GNNpool

In [ ]:
#this cell is only required for collective learning approach
model_type = 'dino_vits8'
feats_dim = 384
extractor = ViTExtractor(model_type, 4, model_dir=pretrained_weights, device=device)
model = GNNpool(feats_dim, 128, 32, K, device, activ).to(device)
#uncomment the line below to load the model for passes of collective learning beyond the first
#model.load_state_dict(torch.load('./modelCVC.pt', map_location=torch.device(device)))
opt = optim.AdamW(model.parameters(), lr=0.0001)
model.train()
kkk = 0
total_iou = 0
imgcount = 1
dir_name = 'CVC_CL_pass1'
dirpath = os.path.join(out_dir,dir_name)
for image, true_mask in tqdm(zip(images, masks)):
    kkk += 1
    filename = image.split('/')[-1].split('.')[0]
    filetype = image.split('/')[-1].split('.')[-1]

    if 'tif' in filetype:
      from libtiff import TIFF
      image = asarray(TIFF.open(image).read_image())
    else:
      image = asarray(Image.open(image))
    true_mask = asarray(Image.open(true_mask).convert('L'))

    image, true_mask = cts(image, true_mask)
    true_mask = np.where(true_mask ==255, 1, 0).astype(np.uint8)

    if 'tif' in filetype:
      image_tensor, image = util.load_data_img1(image, res)
    else:
      image_tensor, image = util.load_data_img(image, res)
    kkk += 1
    # Extract deep features, from the transformer and create an adj matrix
    F = deep_features(image_tensor, extractor, layer, facet, bin=log_bin, device=device)
    W = util.create_adj(F, cut, alpha)
        ###########################################################
        # Data to pytorch_geometric format
    node_feats, edge_index, edge_weight = util.load_data(W, F)
    data = Data(node_feats, edge_index, edge_weight).to(device)
        ##########################################################################################
        # GNN pass
        ##########################################################################################
    for _ in range(5):
        opt.zero_grad()
        A, S = model(data, torch.from_numpy(W).to(device))
        loss = model.loss(A, S)
        loss.backward()
        opt.step()
        # polled matrix (after softmax, before argmax)
    S = S.detach().cpu()
    S = torch.argmax(S, dim=-1)
        ##########################################################################################
        # Post-processing Connected Component/bilateral solver
        ##########################################################################################
    mask0, S = util.graph_to_mask(S, cc, stride, image_tensor, image)
        # apply bilateral solver
    if bs:
        #util.save_or_show([image, np.where(mask0==True, 255,0).astype(np.uint8), util.apply_seg_map(image, mask0, 0.5),true_mask*255], filename, dirpath1 ,save)
        mask0 = bilateral_solver_output(image, mask0,sigma_spatial=11, sigma_luma=5, sigma_chroma=5)[1]
    mask0 = np.where(mask0==True, 1,0).astype(np.uint8)

    try:
      cur_iou = iou(mask0, true_mask)
      cur_iou1 = iou(1-mask0, true_mask)
      if cur_iou < cur_iou1:
        cur_iou = cur_iou1
        mask0 = 1 - mask0
      total_iou += cur_iou
      print(total_iou/imgcount, imgcount,cur_iou)
      imgcount  = imgcount +1
    except:
      pass
    util.save_or_show([image, mask0*255, util.apply_seg_map(image, mask0, 0.5),true_mask*255], filename, dirpath ,save)
torch.save(model.state_dict(), 'modelCVC.pt')


In [ ]:
acc = segment.GNN_seg('./modelCVC.pt',mode, cut, alpha, epochs, K, pretrained_weights, images, masks, out_dir, save, cc, bs, log_bin, res, facet, layer, stride,device, iou,cts, activ)

#modelName should be './model.pt' for non-CL runs